In [1]:
import gradio as gr
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
import cv2
import numpy as np
import math
import random
import bisect
import operator
import matplotlib.pyplot as plt
import copy
import sys
import os
import shutil
import datetime
import time


/home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.12.self_attn.v_proj.bias', 'vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.2.layer_norm1.weight', 'vision_model.encoder.layers.19.self_attn.v_proj.weight', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.6.self_attn.k_proj.bias', 'vision_model.encoder.layers.14.mlp.fc2.weight', 'vision_model.encoder.layers.6.self_attn.q_proj.bias', 'vision_model.encoder.layers.22.mlp.fc1.bias', 'vision_model.encoder.layers.16.self_attn.k_proj.bias', 'vision_model.encoder.layers.23.mlp.fc2.weig

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [3]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

In [4]:

vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [5]:
def func(latents,text_embeddings):
    
    num_inference_steps = 50            # Number of denoising steps

    guidance_scale = 7.5                # Scale for classifier-free guidance

    #############################################

    uncond_input = tokenizer(
        [""] * batch_size, padding="max_length", return_tensors="pt"
    )
    with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

    ###############################################

    embeddings = torch.cat([uncond_embeddings, text_embeddings])

    #############################################

    scheduler.set_timesteps(num_inference_steps)

    ##########################################

    latents_1 = latents * scheduler.init_noise_sigma

    #############################################



    for t in tqdm(scheduler.timesteps):
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input_a = torch.cat([latents_1] * 2)

      latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

    ###############################################
    # scale and decode the image latents with vae
    latents_3 = 1 / 0.18215 * latents_1

    with torch.no_grad():
      image = vae.decode(latents_3).sample


    ##########################################

    image_1 = (image / 2 + 0.5).clamp(0, 1)
    image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image_2 * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    result_image = pil_images[0]
    
    return result_image


In [20]:
#gaの関数

mutation = 0 #突然変異の個体数
population = 9 #個体数
show_image_x = 3
show_image_y = 3
mutation = 0 #突然変異の個体数

if show_image_x*show_image_y!=population or population>9:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)
    
gene_length = 64
elite = 0 #エリートの数
initializa_txt_num = 70 #初期化個体においてtxtのベクトルをどれくらい元から変異させるか
batch_size = 1
height = 512                        # default height of Stable Diffusion
width = 512                        # default width of Stable Diffusion
image_mutation_rate = 400
text_mutation_rate = 6000
adapt = 3 #選択個体の選ばれやすさ

if elite+mutation>population:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)

#seed値を複数個用意
def initialize_gene():
    arr = []
    for i in range(population):
        embeddings = iec_text_embeddings.clone()
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            embeddings[0][a][b] = np.random.randn()
        arr.append([latents,embeddings])    
    return arr


def save_image(diffusion_images,epoch,folder):
    os.mkdir(folder+'/epoch'+str(epoch))
    for i in range (len(diffusion_images)):
        diffusion_images[i].save(folder+"/epoch"+str(epoch)+"/"+str(i)+".png")
        # ログ
        with open(folder+"/epoch"+str(epoch)+"/"+str(i)+"text.txt", 'a') as f:
            print("epoch", iec_live_epoch, file=f)
        # ログ
        with open(folder+"/epoch"+str(epoch)+"/"+str(i)+"text.txt", 'a') as f:
            print("epoch", iec_live_epoch, file=f)


In [21]:
FOREIGN = 2
mutation_rate = 1.0
def evolve(selected,genes):
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
        print("epoch", iec_live_epoch, file=f)
    global evolve_explanation
    evolve_explanation = ""
    evolved_genes,selected_genes = [],[]
    for i in range(population):
        if selected[i]:
            selected_genes.append(genes[i])
    delta = population - len(selected_genes)
    x = max(0,delta-FOREIGN)
    if selected_genes:
        for i in range(x):
            # ログ
            with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                print("population", i, file=f)
            evolved_genes.extend([mutate(simple_crossover(selected_genes,selected),mutation_rate)])
    else:
        return initialize_gene()
    x = min(FOREIGN,delta)
    for i in range(population):
        if selected[i]:
            # ログ
            with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                print("population", i,"mutation", file=f)
    evolved_genes.extend([mutate(selected_genes[i],mutation_rate) for i in range(len(selected_genes))])
    for i in range(x):
        # ログ
        with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
            print("population", i,"initialize", file=f)
        embeddings = iec_text_embeddings.clone()
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            embeddings[0][a][b] = np.random.randn()
        evolved_genes.append([latents,embeddings])  

                    
    if len(evolved_genes)!=population:
        print('Error: evolve error', file=sys.stderr)
        sys.exit(1)
    return evolved_genes

def simple_crossover(gene_population,selected):
    p1 = random.randint(0,len(gene_population)-1)
    p2 = random.randint(0,len(gene_population)-1)
    gene_cnt = 0
    for i in range(population):
        if selected[i]:
            gene_cnt += 1
            if p1==gene_cnt-1:
                # ログ
                with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                    print("crossover1", i, file=f)
            if p2==gene_cnt-1:
                # ログ
                with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                    print("crossover2", i, file=f)
    gene1 = gene_population[p1]
    gene2 = gene_population[p2]
    return crossover(gene1,gene2)

def crossover(gene1,gene2):
    new_gene = copy.deepcopy(gene1)
    gene3 = copy.deepcopy(gene2)
    for j in range(4):
        for k in range(64):
            for l in range(64):
                a = random.randrange(2)
                if a==0:
                    new_gene[0][0][j][k][l] = gene3[0][0][j][k][l]
    for j in range(77):
        for k in range(768):
            new_gene[1][0][j][k] = gene3[1][0][j][k]
    return new_gene

def mutate(individual,mutation_rate = 1.0):
    new_gene = copy.deepcopy(individual)
    for j in range(4):
        for k in range(64):
            for l in range(64):
                a = random.randrange(image_mutation_rate)
                if a==0:
                    value = individual[0][0][j][k][l]+np.random.randn()
                    if value >= 5.00:
                        value = 4.99
                    elif value <=-5.00:
                        value = -4.99
                    new_gene[0][0][j][k][l] = value
    for j in range(77):
        for k in range(768):
            a = random.randrange(text_mutation_rate)
            if a==0:
                value = individual[1][0][j][k]+np.random.randn()
                if value >= 5.00:
                    value = 4.99
                elif value <=-5.00:
                    value = -4.99
                new_gene[1][0][j][k] = value
    return new_gene

In [22]:
import random
import PIL
from PIL import Image,ImageTk
iec_live_epoch=-1
conv_live_epoch=-1
practice_epoch = -1
iec_selected_image_id = 0
conv_selected_image_id = 0
iec_selected = []
for i in range(population):
    iec_selected.append(0)


#本番、２回目以降のiecで画像を更新
def generate_iec_images(*iec_selected_image):
    global iec_live_epoch,iec_selected
    for i in range(population):
        if iec_selected_image[i]:
            iec_selected[i]=1
        else:
            iec_selected[i]=0
    iec_live_epoch += 1
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_live_epoch, 
              "selected population", iec_selected, file=f)
    #進化
    global genes
    genes = evolve(iec_selected,genes)
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_live_epoch, 
              "evolve explanation", evolve_explanation, file=f)
    #画像生成
    iec_images = []
    iec_images.append("試行回数: "+str(iec_live_epoch))
    with autocast("cuda"):
        for i in range(population):
            iec_image = func(genes[i][0],genes[i][1])
            iec_images.append(iec_image)
    save_image(iec_images[1:],iec_live_epoch,iec_image_save_path)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "finished generaging image",file=f)
    return iec_images
#本番、２回目以降のconvで画像を更新
def generate_conv_images(input_prompt):
    global conv_live_epoch
    conv_live_epoch += 1
    
     # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "conv_epoch", conv_live_epoch,
              "input_prompt",input_prompt, file=f)
    text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        conv_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    conv_images = []
    conv_images.append("試行回数: "+str(conv_live_epoch))
    for i in range(population):
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        with autocast("cuda"):
            conv_image = func(latents,conv_text_embeddings)
        conv_images.append(conv_image)
    save_image(conv_images[1:],conv_live_epoch,conv_image_save_path)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "finished generaging image",file=f)
    return conv_images
# 本番、一回目のiec画像生成
def generate_iec_initial_images(input_prompt):
    iec_images = []
    global iec_live_epoch
    #間違って押したときは、さっき表示されていた画像が表示される
    if iec_live_epoch!=-1:
        iec_images.append("試行回数: "+str(iec_live_epoch))
        for i in range(population):
            img = Image.open(iec_image_save_path+'/epoch'+str(iec_live_epoch)+'/' + str(i) + '.png')
            iec_images.append(img)
        return iec_images
    iec_live_epoch += 1
    iec_images.append("試行回数: "+str(iec_live_epoch))
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "iec_epoch", iec_live_epoch,file=f)
    text_input = tokenizer([input_prompt], padding="max_length", 
    max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    global iec_text_embeddings
    with torch.no_grad():
        iec_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    global genes
    genes = initialize_gene()
    with autocast("cuda"):
        for i in range(population):
            iec_image = func(genes[i][0],genes[i][1])
            iec_images.append(iec_image)
    save_image(iec_images[1:],iec_live_epoch,iec_image_save_path)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "finished generaging image",file=f)
    with open('./user_experiment_data/'+experiment_id+'/second_gene.txt', 'a') as f:
        print(genes,file=f)
    return iec_images
# 本番、一回目のconv画像生成
def generate_conv_initial_images(input_prompt):
    conv_images = []
    global conv_live_epoch,conv_text_embeddings
    #間違って押したときは、さっき表示されていた画像が表示される
    if conv_live_epoch!=-1:
        conv_images.append("試行回数: "+str(conv_live_epoch))
        for i in range(population):
            img = Image.open(conv_image_save_path+'/epoch'+str(conv_live_epoch)+'/' + str(i) + '.png')
            conv_images.append(img)
        return conv_images
    conv_live_epoch += 1
    conv_images.append("試行回数: "+str(conv_live_epoch))
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "conv_epoch", conv_live_epoch, "input_prompt",
              input_prompt, file=f)
    text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        conv_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    
    for i in range(population):
        seed_here = random.randrange(1000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        with autocast("cuda"):
            conv_image = func(latents,text_embeddings)
        conv_images.append(conv_image)
    save_image(conv_images[1:],conv_live_epoch,conv_image_save_path)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/log.txt', 'a') as f:
        print(time.time(), "finished generaging image",file=f)
    return conv_images
#練習用の画像を返す
def generate_fake_images():
    global practice_epoch
    practice_epoch+=1
    images = []
    images.append("試行回数: "+str(practice_epoch))
    for i in range(population):
        image = Image.open("sample_image/iec/epoch"+str(practice_epoch)+"/"+str(i)+".png")
        images.append(image)
    return images
def generate_initial_fake_images():
    global practice_epoch
    practice_epoch=0
    images = []
    images.append("試行回数: "+str(practice_epoch))
    for i in range(population):
        image = Image.open("sample_image/iec/epoch"+str(practice_epoch)+"/"+str(i)+".png")
        images.append(image)
    return images
def decide_iec_image(index):
    global iec_selected_image_id
    iec_selected_image_id = index
def decide_conv_image(index):
    global conv_selected_image_id
    conv_selected_image_id = index
    
#結果画像の表示
def show_result_image():
    iec_image = Image.open(iec_image_save_path
                           +'/epoch'+str(iec_live_epoch)+'/' + str(iec_selected_image_id) + '.png')
    conv_image = Image.open(conv_image_save_path+'/epoch'+
                            str(conv_live_epoch)+'/' + str(conv_selected_image_id) + '.png')
    return [iec_image,conv_image]
def reading_text_file(file_name):
    file = open(file_name, 'r')
    text_data = file.read()
    return text_data
with gr.Blocks() as demo:
    with gr.Tab("練習 1"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/iec_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください")
                output.append(gr.Textbox("試行回数: "+str(practice_epoch),label=None))
                first_generate_btn = gr.Button("初期化")
                continue_btn = gr.Button("続ける")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像"),
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
                        population_check.append(gr.Checkbox(label="👍"))
        first_generate_btn.click(fn=generate_initial_fake_images, inputs=None, outputs=output)
        continue_btn.click(fn=generate_fake_images, inputs=None, outputs=output)
    with gr.Tab("練習 2"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/conv_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください")
                output.append(gr.Textbox("試行回数: "+str(practice_epoch),label=None))
                first_generate_btn = gr.Button("初期化")
                continue_btn = gr.Button("続ける")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像"),
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
        first_generate_btn.click(fn=generate_initial_fake_images, inputs=None, 
                                 outputs=output)
        continue_btn.click(fn=generate_fake_images, inputs=None, outputs=output)
    with gr.Tab("本番 1"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/iec_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください")
                output.append(gr.Textbox("試行回数: "+str(iec_live_epoch),label=None))
                first_generate_btn = gr.Button("初期化")
                continue_btn = gr.Button("続ける")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
                        population_check.append(gr.Checkbox(label="👍"))
                        
        first_generate_btn.click(fn=generate_iec_initial_images, inputs=input_prompt, outputs=output)
        continue_btn.click(fn=generate_iec_images, inputs=population_check, outputs=output)
        end_btn.click(fn=decide_iec_image,inputs=iec_select_dropdown,outputs=None)
    with gr.Tab("本番 2"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/conv_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください")
                output.append(gr.Textbox("試行回数: "+str(conv_live_epoch),label=None))
                first_generate_btn = gr.Button("初期化")
                continue_btn = gr.Button("続ける")
                conv_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
        first_generate_btn.click(fn=generate_conv_initial_images, inputs=input_prompt, outputs=output)
        continue_btn.click(fn=generate_conv_images, inputs=input_prompt, outputs=output)
        end_btn.click(fn=decide_conv_image,inputs=conv_select_dropdown,outputs=None)
    with gr.Tab("結果"):
        show_btn = gr.Button("結果を表示する")
        with gr.Row():
            selected_images = [gr.Image(label="実験1").style(height=200,width=200),gr.Image(label="実験2").style(height=200,width=200)]
        show_btn.click(fn=show_result_image,inputs=None,outputs=selected_images)
if __name__ == "__main__":
    #被験者のidを振る
    experiment_time_delta = datetime.timedelta(hours=9)
    experiment_JST = datetime.timezone(experiment_time_delta, 'JST')
    global experiment_id,iec_image_save_path,conv_image_save_path
    experiment_now = datetime.datetime.now(experiment_JST)
    experiment_id = experiment_now.strftime('%Y%m%d%H%M%S')
    iec_image_save_path='./user_experiment_data/'+experiment_id+'/iec'
    conv_image_save_path='./user_experiment_data/'+experiment_id+'/conv'
    #ディレクトリ初期化
    os.mkdir('./user_experiment_data/'+experiment_id)
    os.mkdir(iec_image_save_path)
    os.mkdir(conv_image_save_path)
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7875
Running on public URL: https://2474321d-d9bb-4012.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


100%|██████████| 50/50 [00:05<00:00,  9.10it/s]


In [45]:

print(r)


SyntaxError: EOL while scanning string literal (3627838035.py, line 1)

In [9]:
print(random.randint(0,1))

1
